In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Business Understanding**

Brain tumors in particular are very difficult to diagnose from an MRI image, and artificial intelligence methods of identifying and classifying tumors are oftentimes more accurate than manual identification by a radiologist. That is why the development of neural networks and other AI processes for tumor classification is so valuable and important.

The survival rate for patients diagnosed with a brain tumor is around 35%. This survival rate could be increased if tumors could be identified earlier and more accurately, which AI methods could help with. Additionally, in third world countries, seasoned neurosurgeons (a neurosurgeon is required to make the diagnoses from looking at the MRI) are hard to come by, so a machine learning tool (Decision Support Tool) which could accurately identify tumors would be of great value in these developing nations. This decision support tool would be beneficial to the health industry, and the target audience would be Doctors without Borders, an organization which sends doctors from the US to developing countries to help improve their healthcare. 

In [16]:
# Set random state for numpy operations
from numpy.random import seed
seed(42)
# Set random state for tensorflow operations
from tensorflow.random import set_seed
set_seed(42)
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
import cv2
import PIL

In [3]:

def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [4]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=([0.6,1]),
                                   rotation_range=10,                           
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=([0.6,1]),
                                   rotation_range=10,
                                   brightness_range=([0.6,1.5]),
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing',
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary',
                                    subset='validation')

## **Data Understanding**

In [ ]:
# Visualize (code from https://github.com/austint1121/OES-PneumoniaClassification/blob/main/Final_Notebook.ipynb)
train_batch = train_generator.next()
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    
for i in range(10):
    # Load image into numpy array and re-scale
    img = np.array(train_batch[0][i] * 255, dtype='uint8')
    ax = axes[i // 5, i % 5]
    ax.imshow(img)
fig.suptitle('Training Images')
plt.tight_layout()
plt.show()

In [ ]:
for i in range(10):
    ax = axes[i // 5, i % 5]
    for image, label in train_batch:
        if label == 1:
            ax.imshow(np.array(image[i] * 255, dtype='uint8'))
fig.suptitle('No Tumor')
plt.tight_layout()
plt.show()

## **Taking a Look at a few different individual images**

In [ ]:
tumor1 = PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/gg (108).jpg')
tumor1

In [ ]:
# Figuring out the number of color channels
tumor1.mode

## **Taking a look at the class imbalance**

First looking at training data imbalance

In [18]:
train_generator.class_indices

In [17]:
train_generator.classes

In [19]:
train_tumors = pd.DataFrame(train_generator.classes)
train_values = train_tumors.value_counts()
train_values

The ratio of images with tumors to those without is 2327:372, or 6.255:1.

In [20]:
train_tumors.rename(columns={0:'Tumor/No Tumor'}, inplace=True)

In [21]:
train_tumors[train_tumors['Tumor/No Tumor'] == 0]

In [22]:
train_no_tumor = len(train_tumors[train_tumors['Tumor/No Tumor'] == 1])
train_tumor = len(train_tumors[train_tumors['Tumor/No Tumor'] == 0])

In [23]:
#plt.figure(figsize=(10,6))
#sns.set(font_scale=1.4)
#sns.barplot(tumors.index, tumors.values)
#plt.ylabel("Number of Images")
#plt.title('Distribution of Brain MRIs with and without Tumor');

fig, ax = plt.subplots(figsize=(10,8))
ax.bar(x=['No Tumor', 'Tumor'],height = [train_no_tumor, train_tumor])
ax.set(xlabel='', ylabel='Number of Images', title='Distribution of Brain MRIs with and without Tumor');

Now taking a look at the test data imbalance

In [ ]:
test_tumors = pd.DataFrame(test_generator.classes)
test_values = test_tumors.value_counts()
test_values

the ratio of images with tumors to those without is 289:105, or 2.75:1

In [ ]:
test_tumors.rename(columns={0:'Tumor/No Tumor'}, inplace=True)

In [ ]:
test_no_tumor = len(test_tumors[test_tumors['Tumor/No Tumor'] == 1])
test_tumor = len(test_tumors[test_tumors['Tumor/No Tumor'] == 0])

In [ ]:

fig, ax = plt.subplots(figsize=(10,8))
ax.bar(x=['No Tumor', 'Tumor'],height = [test_no_tumor, test_tumor])
ax.set(xlabel='', ylabel='Number of Images', title='Distribution of Brain MRIs with and without Tumor in Testing Data');

## **Baseline CNN Model**

In [ ]:
# Building the first baseline model; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
baseline = keras.Sequential()
baseline.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline.add(layers.MaxPooling2D(2,2))
baseline.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline.add(layers.MaxPooling2D(2,2))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(128, activation='relu'))
baseline.add(layers.Dense(1, activation='sigmoid'))

baseline.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
baseline_results = baseline.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(baseline_results)

### **Analysis of Model**

Looking at the above graphs, it is obvious that this first baseline cnn model is overfitting; accuracy for training data ends up at around 94%, whereas testing data ends up at around 65%. Additionally, the loss for testing data is fairly high; for training the loss ends up at 15%, and for testing it ends up at 69%. In the next model iteration, I will add another dense layer, which will hopefully help the model pick up on more patterns, and some dropout layers for a form of regularization.

## **Adding another Dense layer and Dropout layers**

In [ ]:
# Adding another dense layer and a couple of dropout layers; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
layers_drop = keras.Sequential()
layers_drop.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
layers_drop.add(layers.MaxPooling2D(2,2))
layers_drop.add(layers.Conv2D(64, (3,3), activation='relu'))
layers_drop.add(layers.MaxPooling2D(2,2))

layers_drop.add(layers.Flatten())
layers_drop.add(layers.Dense(128, activation='relu'))
layers_drop.add(layers.Dropout(0.3))
layers_drop.add(layers.Dense(64, activation='relu'))
layers_drop.add(layers.Dropout(0.3))
layers_drop.add(layers.Dense(1, activation='sigmoid'))

layers_drop.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
# Fitting the model
layers_drop_results = layers_drop.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(layers_drop_results)

### **Analysis of Model**

In this iteration, training accuracy ends up at 93%, and testing ends up at 73%, so the model is overfitting, but less so than the baseline model. As for loss, training loss is 14% and testing loss is 65%, which is not drastically different from the last model. Adding another layer and dropout layers helped decrease overfitting. In the next model iteration I am going to account for the class imabalance, and the added layer and dropout layers might perform better in this iteration.

## **Accounting for class imbalance**

In [ ]:
# Accounting for class imbalance; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

class_ld = keras.Sequential()
class_ld.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_ld.add(layers.MaxPooling2D(2,2))
class_ld.add(layers.Conv2D(64, (3,3), activation='relu'))
class_ld.add(layers.MaxPooling2D(2,2))

class_ld.add(layers.Flatten())
class_ld.add(layers.Dense(128, activation='relu'))
class_ld.add(layers.Dropout(0.3))
class_ld.add(layers.Dense(64, activation='relu'))
class_ld.add(layers.Dropout(0.3))
class_ld.add(layers.Dense(1, activation='sigmoid'))

class_ld.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
# Fitting the model
class_ld_results = class_ld.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_ld_results)

### **Analysis of Model**

In this model iteration, training accuracy was about 89% and testing accuracy is about 57%, so the model is still overfitting compared to the last model. Loss for training is at around 51% and testing loss is around 99%. In terms of acuracy and loss, the model is doing worse than the previous model. However, recall has increased significantly, so it seems that adding class weights is beneficial to the model, even though it requires further tuning.

## **Adding another Convolution layer**

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
class_con = keras.Sequential()
class_con.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_con.add(layers.MaxPooling2D(2,2))
class_con.add(layers.Conv2D(64, (3,3), activation='relu'))
class_con.add(layers.MaxPooling2D(2,2))
class_con.add(layers.Conv2D(128, (3,3), activation='relu'))
class_con.add(layers.MaxPooling2D(2,2))

class_con.add(layers.Flatten())
class_con.add(layers.Dense(128, activation='relu'))
class_con.add(layers.Dropout(0.3))
class_con.add(layers.Dense(64, activation='relu'))
class_con.add(layers.Dropout(0.3))
class_con.add(layers.Dense(1, activation='sigmoid'))

class_con.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, 
          1:6.255}

In [ ]:
class_con_results = class_con.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_con_results)

### **Analysis of Model**

Training accuracy is at 89% while testing accuracy is at 50%, so model is still overfitting, slightly worse than the previous model. Training loss is 50%, and testing loss is 93%; training loss is 50% and testing loss is 93%, which is similar to the loss values of the previous model. In the next model iteration, I will see if changing the dimensions of the pooling layer will improve the model.


## **Adjusting the Pooling Strategy**

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_pool = keras.Sequential()
class_pool.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_pool.add(layers.MaxPooling2D(2,2))
class_pool.add(layers.Conv2D(64, (3,3), activation='relu'))
class_pool.add(layers.MaxPooling2D(3,3))
class_pool.add(layers.Conv2D(128, (3,3), activation='relu'))
class_pool.add(layers.MaxPooling2D(5,5))

class_pool.add(layers.Flatten())
class_pool.add(layers.Dense(128, activation='relu'))
class_pool.add(layers.Dropout(0.3))
class_pool.add(layers.Dense(64, activation='relu'))
class_pool.add(layers.Dropout(0.3))
class_pool.add(layers.Dense(1, activation='sigmoid'))

class_pool.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
class_pool_results = class_pool.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_pool_results)

### **Analysis of Model**

Training accuracy is at about 89%, and testing accuracy is at about 48%, so model is still overfitting quite a bit. Training loss is 48%, and testing loss is 101%; testing loss has increased quite a lot since the last model. Maybe increasing the pooling matrix is not beneficial to the model,so I will return the pooling strategy to all be (2,2) matrices and introduce padding, as this may help.. In this next model, I will introduce some padding to reduce image loss, to see if this improves model.

## **Model with Padding**

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
class_pad = keras.Sequential()
class_pad.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(200,200,3)))
class_pad.add(layers.MaxPooling2D(2,2))
class_pad.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
class_pad.add(layers.MaxPooling2D(2,2))
class_pad.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
class_pad.add(layers.MaxPooling2D(2,2))

class_pad.add(layers.Flatten())
class_pad.add(layers.Dense(128, activation='relu'))
class_pad.add(layers.Dropout(0.3))
class_pad.add(layers.Dense(64, activation='relu'))
class_pad.add(layers.Dropout(0.3))
class_pad.add(layers.Dense(1, activation='sigmoid'))

class_pad.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
early_stop = [EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
class_pad_results = class_pad.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_pad_results)

### **Analysis of Model**

Training accuracy is 89%, testing accuracy is 40%, so the model is overfitting more so than in the "Adding another Convolutional layer" model, which is identical to this one excpet for the padding. Training loss is 50% and testing loss is 129%; testing loss increased significantly from the "Adding another Convolutional Layer" model. It looks like this strategy of padding is making the model worse, so I will remove it. Additionally, because loss is a big problem, I am going to try decreasing network size and increasing dropout layers.

## **Model without Padding, a deleted Dense layer, Early Stopping, and more Epochs**

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_ee = keras.Sequential()
class_ee.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_ee.add(layers.MaxPooling2D(2,2))
class_ee.add(layers.Conv2D(64, (3,3), activation='relu'))
class_ee.add(layers.MaxPooling2D(2,2))
class_ee.add(layers.Conv2D(128, (3,3), activation='relu'))
class_ee.add(layers.MaxPooling2D(2,2))

class_ee.add(layers.Flatten())
class_ee.add(layers.Dense(128, activation='relu'))
class_ee.add(layers.Dropout(0.3))
class_ee.add(layers.Dense(1, activation='sigmoid'))

class_ee.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [5]:
early_stop2 = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
class_ee_results = class_ee.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=50,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_ee_results)

### **Analysis of Model**

Training accuracy of the best epoch (epoch4) is around 88%, and testing accuracy is around 50%. Training loss is around 60% and testing loss is around 113%. Testing accuracy has improved by 10% since the last model, and testing loss has decreased by about sixteen percentage points, so removing a dense layer seems to be a slight improvement. I will try batch normalization to see if this helps train the model faster.

## **Model with Batch Normalization** 

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_n = keras.Sequential()
class_n.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_n.add(layers.BatchNormalization())
class_n.add(layers.MaxPooling2D(2,2))
class_n.add(layers.Conv2D(64, (3,3), activation='relu'))
class_n.add(layers.BatchNormalization())
class_n.add(layers.MaxPooling2D(2,2))
class_n.add(layers.Conv2D(128, (3,3), activation='relu'))
class_n.add(layers.BatchNormalization())
class_n.add(layers.MaxPooling2D(2,2))

class_n.add(layers.Flatten())
class_n.add(layers.Dense(128, activation='relu'))
class_n.add(layers.Dropout(0.3))
class_n.add(layers.Dense(1, activation='sigmoid'))

class_n.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
class_n_results = class_n.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                         callbacks=early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_n_results)

The best model from epoch 14 had a training accuracy of around 94%, a testing accuracy of around 70%, a training loss of 30%, and a testing loss of around 84%. Accuracy and loss for both training and testing data is much improved since the last model and it is less overfit, so batch normalization is definitely an improvement. 
Because batch normalization makes the network more stable, it is possible to use larger learning rates, which could potentially help the model reach optimal accuracy and minimal loss more quickly, so that is what I will try next.

## **Using a Bigger Learning Rate since I am using Batch Normalization**

In [ ]:
adam_mlr = keras.optimizers.Adam(epsilon=0.01)

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_na = keras.Sequential()
class_na.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_na.add(layers.BatchNormalization())
class_na.add(layers.MaxPooling2D(2,2))
class_na.add(layers.Conv2D(64, (3,3), activation='relu'))
class_na.add(layers.BatchNormalization())
class_na.add(layers.MaxPooling2D(2,2))
class_na.add(layers.Conv2D(128, (3,3), activation='relu'))
class_na.add(layers.BatchNormalization())
class_na.add(layers.MaxPooling2D(2,2))

class_na.add(layers.Flatten())
class_na.add(layers.Dense(128, activation='relu'))
class_na.add(layers.Dropout(0.3))
class_na.add(layers.Dense(1, activation='sigmoid'))

class_na.compile(loss='binary_crossentropy',
                optimizer=adam_mlr,
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
class_na_results = class_na.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                         callbacks=early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_na_results)

In the best epoch of the model (epoch 20) training accuracy is 95% while testing accuracy is 74%. Training loss is 33%, while testing loss is 55%. Testing accuracy is higher by four percentage points than the last model, and loss has decreased by about 30%! Additionally, testing recall is 93%, which is important for the context of this problem. Since the last epoch was the best, it might be that the model has not yet reached optimal accuracy and minimal loss, so I am going to increase the number of epochs in the next model iteration.

## **Model with Batch Normalization, bigger learning rate, and more Epochs**

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_nae = keras.Sequential()
class_nae.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_nae.add(layers.BatchNormalization())
class_nae.add(layers.MaxPooling2D(2,2))
class_nae.add(layers.Conv2D(64, (3,3), activation='relu'))
class_nae.add(layers.BatchNormalization())
class_nae.add(layers.MaxPooling2D(2,2))
class_nae.add(layers.Conv2D(128, (3,3), activation='relu'))
class_nae.add(layers.BatchNormalization())
class_nae.add(layers.MaxPooling2D(2,2))

class_nae.add(layers.Flatten())
class_nae.add(layers.Dense(128, activation='relu'))
class_nae.add(layers.Dropout(0.3))
class_nae.add(layers.Dense(1, activation='sigmoid'))

class_nae.compile(loss='binary_crossentropy',
                optimizer=adam_mlr,
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
class_nae_results = class_nae.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=50,
                                         callbacks=early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_nae_results)

## **Model with Batch Normalization and more Dropout Layers**

In [ ]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_nd = keras.Sequential()

class_nd.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_nd.add(layers.BatchNormalization())
class_nd.add(layers.MaxPooling2D(2,2))
class_nd.add(layers.Dropout(0.25))

class_nd.add(layers.Conv2D(64, (3,3), activation='relu'))
class_nd.add(layers.BatchNormalization())
class_nd.add(layers.MaxPooling2D(2,2))
class_nd.add(layers.Dropout(0.25))

class_nd.add(layers.Conv2D(128, (3,3), activation='relu'))
class_nd.add(layers.BatchNormalization())
class_nd.add(layers.MaxPooling2D(2,2))
class_nd.add(layers.Dropout(0.25))

class_nd.add(layers.Flatten())
class_nd.add(layers.Dense(128, activation='relu'))
class_nd.add(layers.Dropout(0.4))
class_nd.add(layers.Dense(1, activation='sigmoid'))

class_nd.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
class_nd_results = class_nd.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                          callbacks=early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(class_nd_results)

Training accuracy of the best epoch (epoch 1) is 91% while testing accuracy is 73%. Training loss is 56% while testing loss is 82%. This model is similar to the last one, except for the fact that training loss is significantly increased. The dropout layers added after each max pooling step may not be particularly beneficial to the model.

## **Going Back to Baseline, but adding Batch Normalization Layers**

In [ ]:
# Building the first baseline model; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
baseline_n = keras.Sequential()
baseline_n.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline_n.add(layers.BatchNormalization())
baseline_n.add(layers.MaxPooling2D(2,2))

baseline_n.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline_n.add(layers.BatchNormalization())
baseline_n.add(layers.MaxPooling2D(2,2))

baseline_n.add(layers.Flatten())
baseline_n.add(layers.Dense(128, activation='relu'))
baseline_n.add(layers.BatchNormalization())
baseline_n.add(layers.Dense(1, activation='sigmoid'))

baseline_n.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
baseline_n_results = baseline_n.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks=early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(baseline_n_results)

In [ ]:
# Getting the predicted labels for all images in testing data
baseline_n_pred = baseline_n.predict(test_generator)
len(baseline_n_pred)

In [ ]:
# Rounding the predictions and making sure all values are integer type so that they can be directly compared to the true labels, which are all ints
base_npred_round = np.round(baseline_n_pred)
npred_int = []
for entry in base_npred_round:
    npred_int.append(int(entry))
npred_int

In [ ]:
# Looking at true labels
test_generator.classes

In [ ]:
# I want to find out all of the indices of the incorrectly classified images, as well as their label
wrong_index = []
wrong_entry = []
for index, entry in enumerate(npred_int):
    if entry != test_generator.classes[index]:
        wrong_index.append(index)
        wrong_entry.append(entry)
wrong_index_entry = list(zip(wrong_index, wrong_entry))

In [ ]:
# Getting a list of all file names of incorrectly classified tumors, so that I can view the images
fnames = test_generator.filenames 
wrong_fnames = []
for i in wrong_index:
    wrong_fnames.append(fnames[i])
wrong_fnames

In [ ]:
# Getting the predicted labels for all images in testing data
baseline_n_pred = baseline_n.predict(test_generator)
len(baseline_n_pred)

# Rounding the predictions and making sure all values are integer type so that they can be directly compared to the true labels, which are all ints
base_npred_round = np.round(baseline_n_pred)
npred_int = []
for entry in base_npred_round:
    npred_int.append(int(entry))
npred_int

# Looking at true labels
test_generator.classes

# I want to find out all of the indices of the incorrectly classified images, as well as their label
wrong_index = []
wrong_entry = []
for index, entry in enumerate(npred_int):
    if entry != test_generator.classes[index]:
        wrong_index.append(index)
        wrong_entry.append(entry)
wrong_index_entry = list(zip(wrong_index, wrong_entry))

# Getting a list of all file names of incorrectly classified tumors, so that I can view the images
fnames = test_generator.filenames 
wrong_fnames = []
for i in wrong_index:
    wrong_fnames.append(fnames[i])
wrong_fnames

# A Glioma the network mislabelled as not having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Glioma14.jpg')

In [ ]:
# A Glioma the network mislabelled as not having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Glioma14.jpg')

In [ ]:
# A Glioma the network mislabelled as not having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Glioma2.jpg')

In [ ]:
# A Glioma the network mislabelled as not having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Glioma23.jpg')

In [ ]:
# A Meningioma the network mislabelled as not having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Meningioma10.jpg')

In [ ]:
# A Meningioma the network mislabelled as not having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Meningioma17.jpg')

In [ ]:
# A Meningioma the network correctly identified as having a tumor
PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing/AllTumorsTest/Meningioma19.jpg')

The best epoch of this model has a training accuracy of 93% and a testing accuracy of 80%. It has a training loss of 17% and a testing loss of 54%. This model has much better accuracy and much less loss than the last model, so adding batch normalization really helped.
* look at epoch 10!

## **Going Back to Baseline, but adding Batch Normalization and a Dropout layer**

In [ ]:
# Building the first baseline model; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
baseline_nd = keras.Sequential()
baseline_nd.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline_nd.add(layers.BatchNormalization())
baseline_nd.add(layers.MaxPooling2D(2,2))

baseline_nd.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline_nd.add(layers.BatchNormalization())
baseline_nd.add(layers.MaxPooling2D(2,2))

baseline_nd.add(layers.Flatten())
baseline_nd.add(layers.Dense(128, activation='relu'))
baseline_nd.add(layers.Dropout(0.3))
baseline_nd.add(layers.BatchNormalization())
baseline_nd.add(layers.Dense(1, activation='sigmoid'))

baseline_nd.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
baseline_nd_results = baseline_nd.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks=early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(baseline_nd_results)

The epoch with the best model (epoch 20) had a training accuracy of 94% and a testing accuracy of 76%. Training loss is 16% and testing loss is 54%.Testing accuracy is slightly less than the last model, so further tuning is necessary. Next I will try incorporating the class weights to account for the class imbalance.

## **Going back to Baseline, adding Class Weights**

In [ ]:

base_class = keras.Sequential()
base_class.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
base_class.add(layers.BatchNormalization())
base_class.add(layers.MaxPooling2D(2,2))

base_class.add(layers.Conv2D(64, (3,3), activation='relu'))
base_class.add(layers.BatchNormalization())
base_class.add(layers.MaxPooling2D(2,2))

base_class.add(layers.Flatten())
base_class.add(layers.Dense(128, activation='relu'))
base_class.add(layers.Dropout(0.3))
base_class.add(layers.BatchNormalization())
base_class.add(layers.Dense(1, activation='sigmoid'))

base_class.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
class_weights2 = {0:1,
                 1:6.255}

In [ ]:
base_class_results =base_class.fit_generator(train_generator,
                                        class_weight = class_weights2,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks=early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing loss has a training accuracy of around 86% and a testing accuracy of around 62%, with a training loss of 55% and a testing loss of 76%. Testing recall is 76%. These results are worse than the last model iteration, so it looks like adding class weights did not help in this case.

## **Using the Pre-Trained VGG-19 Weights (this is my FSM and may be my Final Model!)**

In [6]:
from keras.applications.vgg19 import VGG19
cnn_vgg = VGG19(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [7]:
cnn_vgg.summary()


In [24]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
pretrained = keras.Sequential()
pretrained.add(cnn_vgg)
pretrained.add(layers.Flatten())
pretrained.add(layers.Dense(128, activation='relu'))
pretrained.add(layers.Dense(1, activation='sigmoid'))

In [8]:
def Freeze_Pretrained_Base(pretrain, network):
    pretrain.trainable = False
    for layer in network.layers:
        print(layer.name, layer.trainable)
    print(len(network.trainable_weights))

In [25]:
Freeze_Pretrained_Base(cnn_vgg, pretrained)

In [26]:

pretrained.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

pretrained_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(pretrained_results)

### **Analysis of Model**
The epoch with the lowest loss had a training accuracy of ~98% and a loss of ~4%, while the testing data had an accuracy of ~93% and a loss of ~21%! This is the best model yet! Additionally, testing recall is ~95%, which means that false negatives are being minimized. Using the VGG19 pretrained weights was a game changer! However, it would be great if the testing loss could be just a bit lower, so in the next model iteration I will use the SGD optimizer with momentum, since it is known for rapidly decreasing loss.

## **Unfreezing Layers in the Pretrained VGG-19 Network**

In [27]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5c1c2 = keras.Sequential()
b5c1c2.add(cnn_vgg)
b5c1c2.add(layers.Flatten())
b5c1c2.add(layers.Dense(128, activation='relu'))
b5c1c2.add(layers.Dense(1, activation='sigmoid'))

In [11]:
unfreeze = ['block5_conv1', 'block5_conv2']

In [9]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
def Unfreeze_Layers(pretrain, layer_list):
    pretrain.trainable = True
    for layer in  pretrain.layers:
        if layer.name in layer_list:
            layer.trainable = True
        else:
            layer.trainable = False
        
    for layer in pretrain.layers:
        print(layer.name, layer.trainable)
    print(len(pretrain.trainable_weights))

In [12]:
Unfreeze_Layers(cnn_vgg, unfreeze)

In [31]:
b5c1c2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5c1c2_results = b5c1c2.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing los has a trining accuracy of 98% and a testing accuracy of 92%, with a training loss of 5% and a testing loss of 27%. Testing recall is 75%. Other than recall, the results are similar to the previous model; maybe including class weights will help improve the model.

In [33]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
w_b5c1c2 = keras.Sequential()
w_b5c1c2.add(cnn_vgg)
w_b5c1c2.add(layers.Flatten())
w_b5c1c2.add(layers.Dense(128, activation='relu'))
w_b5c1c2.add(layers.Dense(1, activation='sigmoid'))

In [34]:

weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without


w_b5c1c2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

w_b5c1c2_results = w_b5c1c2.fit_generator(train_generator,
                                          class_weight = weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)


**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 98% and a testing accuracy of 94%, with a training loss of 7% and a testing loss of 22%. Testing recall 97%. Overall, because the recall is higher, this model is better than the first model iteration done using the pretrained VGG19 network.

## **Adjusting Class Weights and adding a Dropout Layer**

In [13]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
w2_b5c1c2 = keras.Sequential()
w2_b5c1c2.add(cnn_vgg)
w2_b5c1c2.add(layers.Dropout(0.4))
w2_b5c1c2.add(layers.Flatten())
w2_b5c1c2.add(layers.Dense(128, activation='relu'))
w2_b5c1c2.add(layers.Dense(1, activation='sigmoid'))

In [14]:
Freeze_Pretrained_Base(cnn_vgg, w2_b5c1c2 )

In [15]:
Unfreeze_Layers(cnn_vgg, unfreeze)

In [17]:

weights2 = {0: 1, # TUMOR
          1:4} # NO TUMOR 



w2_b5c1c2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

w2_b5c1c2_results = w2_b5c1c2.fit_generator(train_generator,
                                          class_weight = weights2,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

The epoch with the lowest testing loss has a training accuracy of 98% and a testing accuracy of 94%, with a training loss of 7% and a testing loss of 21%. Testing recall is 99%. This model is better than all iterations using VGG19 thus far, due to the high recall percentage in this epoch, as well as all epochs.

## **Unfreezing more layers of VGG-19 Base, including Adjusted Class Weights**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
w2_b5c1c2 = keras.Sequential()
w2_b5c1c2.add(cnn_vgg)
w2_b5c1c2.add(layers.Dropout(0.4))
w2_b5c1c2.add(layers.Flatten())
w2_b5c1c2.add(layers.Dense(128, activation='relu'))
w2_b5c1c2.add(layers.Dense(1, activation='sigmoid'))

## **Using the Pretrained VGG-19 network again, but with an SGD Optimizer**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
vgg19_sgd = keras.Sequential()
vgg19_sgd.add(cnn_vgg)
vgg19_sgd.add(layers.Flatten())
vgg19_sgd.add(layers.Dense(128, activation='relu'))
vgg19_sgd.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# Make the pretrianed layer untrainable so that during optimization, its weights don't change
cnn_vgg.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in vgg19_sgd.layers:
    print(layer.name, layer.trainable)
    
print(len(vgg19_sgd.trainable_weights))

In [ ]:
sgd_momen = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)



vgg19_sgd.compile(loss='binary_crossentropy',
                optimizer= sgd_momen,
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

vgg19_sgd_results = vgg19_sgd.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 96% and a testing accuracy of 83%, with a training loss of 11% and a testing loss of 33%. Testing recall is 71%. Although the results from this model iteration, using SGD with momentum instead of Adam as an optimizer, they are not as good as the previous model iteration which used Adam, so it looks like Adam is the best optimizer to use in this situation.